In [ ]:
import pandas as pd
import time
import numpy as np
from datetime import datetime

In [ ]:
df=pd.read_csv("dataset.csv")

In [ ]:
df = df[df.text.isna()==False]

In [ ]:
df.shape

In [ ]:
df["target"].value_counts()

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df = df.rename(columns={"sub-title": "subtitle"})

In [ ]:
df['text'] = df['text'].apply(lambda x: x.replace("— GIB © RIPRODUZIONE RISERVATA", ""))
df['text'] = df['text'].apply(lambda x: x.replace("© riproduzione riservata", ""))

In [ ]:
df = df[(df['target']=='maltrattamento') | (df['target']=='aggressione') | (df['target']=='violenza sessuale')].copy()

In [ ]:
df.shape

In [ ]:
# pip install spacy_dbpedia_spotlight

In [ ]:
import spacy_dbpedia_spotlight

nlp = spacy_dbpedia_spotlight.create('it')
print(nlp.pipe_names)

In [ ]:
# pip install SPARQLWrapper

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

In [ ]:
my_types = {"http://dbpedia.org/ontology/Person", "http://dbpedia.org/ontology/Organisation", 
            "http://dbpedia.org/ontology/Building", "http://dbpedia.org/ontology/Place", 
            "http://dbpedia.org/ontology/TimePeriod"}

In [ ]:
all_entities={}

In [ ]:
def extract_entities(text):
    
    types=set()
    entities={}
    
    doc=nlp(text)
    # print("---------------------------------------------------------------")
    # print([(ent.text, ent.kb_id_, ent._.dbpedia_raw_result['@similarityScore']) for ent in doc.ents])
    for ent in doc.ents:
        # print("---------------------")
        # print(ent.text)
        if(float(ent._.dbpedia_raw_result['@similarityScore'])>=0.7):
            # print(ent.kb_id_)
            # print(ent._.dbpedia_raw_result['@support'])
            # print(ent._.dbpedia_raw_result['@similarityScore'])
            key = ent.kb_id_.replace("http://it.dbpedia", "http://dbpedia")
            if(key in entities):
                count=entities[key]
                entities[key]=count+1
            else:
                entities[key]=1
                
            if(key in all_entities):
                count=all_entities[key]
                all_entities[key]=count+1
            else:
                all_entities[key]=1
            
            sparql = SPARQLWrapper("http://dbpedia.org/sparql/")
            sparql.setReturnFormat(JSON)
            
            query = """select distinct ?type where {
                       <""" + key + """>  a/rdfs:subClassOf{0,5} ?type . }
                    """
            sparql.setQuery(query)
            try:
                ret = sparql.queryAndConvert()
                for r in ret["results"]["bindings"]:
                    if("http://dbpedia.org/ontology" in r["type"]["value"]):
                        #print(r["type"]["value"])
                        types.add(r["type"]["value"])
            except Exception as e:
                print(e)
            
            if len(types)==0:
                query = """select distinct ?type where {
                           <""" + key + """> owl:equivalentClass{0,1}/a ?type . }
                        """
                sparql.setQuery(query)
                try:
                    ret = sparql.queryAndConvert()
                    for r in ret["results"]["bindings"]:
                        if("http://dbpedia.org/ontology" in r["type"]["value"]):
                            #print(r["type"]["value"])
                            types.add(r["type"]["value"])
                except Exception as e:
                    print(e)
            
            if len(types)==0:
                query = """select distinct ?type where {
                           <""" + key + """> dbo:wikiPageRedirects/a ?type . }
                        """
                sparql.setQuery(query)
                try:
                    ret = sparql.queryAndConvert()
                    for r in ret["results"]["bindings"]:
                        if("http://dbpedia.org/ontology" in r["type"]["value"]):
                            #print(r["type"]["value"])
                            types.add(r["type"]["value"])
                except Exception as e:
                    print(e)
            
            if len(types)==0:
                query = """select distinct ?type where {
                           <""" + key + """> owl:sameAs{0,1}/a ?type . }
                        """
                sparql.setQuery(query)        
                try:
                    ret = sparql.queryAndConvert()
                    for r in ret["results"]["bindings"]:
                        if("http://dbpedia.org/ontology" in r["type"]["value"]):
                            #print(r["type"]["value"])
                            types.add(r["type"]["value"])
                except Exception as e:
                    print(e)
              
            flag=False
            for t in my_types:
                if t in types:
                    flag=True
            if flag==True:
                if(key in entities):
                    count=entities[key]
                    entities[key]=count+1
                else:
                    entities[key]=1

                if(key in all_entities):
                    count=all_entities[key]
                    all_entities[key]=count+1
                else:
                    all_entities[key]=1
    # print(entities)
    return entities

In [ ]:
df['entities']=df['text'].apply(lambda x: extract_entities(x))

In [ ]:
df['entities']

In [ ]:
def update_score(dict_):
    new_dict={}
    for k in dict_:
        new_dict[k]=(dict_[k]/all_entities[k])
    return new_dict

In [ ]:
df['entities_with_score']=df['entities'].apply(lambda x: update_score(x))

In [ ]:
df['entities_with_score']

In [ ]:
# pip install git+https://github.com/LIAAD/yake

In [ ]:
import yake

In [ ]:
language = "it"
max_ngram_size = 1
deduplication_thresold = 0.7
deduplication_algo = 'jaro'
windowSize = 4
numOfKeywords = 30

file = open("italian")    # INSERT PATH TO THE FILE WITH STOPWORDS
stopword_set = set(file.read().lower().split("\n"))
        
custom_kw_extractor = yake.KeywordExtractor(lan=language, stopwords=stopword_set, n=max_ngram_size, dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)

def extract_keyphrases(x):
    keyphrases={}
    keys=[]
    scores=[]
    keywords = custom_kw_extractor.extract_keywords(x)
    print(keywords)
    for kw in keywords:
        keys.append(kw[0].lower())
        scores.append(kw[1])
    print(scores)
    min_ = min(scores)
    max_ = max(scores)
    if(min_!=max_):
        scores = [1-((num-min_)/(max_-min_)) for num in scores]
        for i in range(5):
            keyphrases[keys[i]]=scores[i]
    else:
        scores = [1]
        keyphrases[keys[0]]=scores[0]
    
    print(scores)
    print(keyphrases)
    return keyphrases

In [ ]:
df['keyphrases'] = df['text'].apply(lambda x: extract_keyphrases(x))

In [ ]:
df['keyphrases']

In [ ]:
keys=set()

def read_keyphrases(x):
    k = list(x.keys())
    for el in k:
        keys.add(el)
    # keys.add(x.keys())
    return keys

In [ ]:
all_keyphrases = df['keyphrases'].apply(lambda x: read_keyphrases(x))

In [ ]:
df['my_entities_with_score'] = df['entities_with_score'].apply(lambda x: str(x)[1:-1])

In [ ]:
df['my_keyphrases'] = df['keyphrases'].apply(lambda x: str(x)[1:-1])

In [ ]:
df.to_csv("enhanced_dataset.csv", index=False)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
from neo4j import GraphDatabase

class NewDb:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        
    def close(self):
        self.driver.close()
        
    def _prova_creazione(self):
        with self.driver.session() as session:
            session.run("CALL n10s.graphconfig.init({handleVocabUris: 'MAP', handleMultival: 'ARRAY', keepLangTag: false, keepCustomDataTypes: true, applyNeo4jNaming: true })")
            session.write_transaction(self._creazione)
            
    @staticmethod
    def _creazione(tx):
        tx.run("""LOAD CSV WITH HEADERS FROM \"file:///enhanced_dataset.csv\" AS line FIELDTERMINATOR \",\" 
                MERGE(c:Document {id: line.id})
                SET c += {url: line.url, title: line.title, text: line.text, publication_date: line.publication_date}
                WITH line, c
                UNWIND split(line.my_entities_with_score, ',') AS dblink 
                WITH DISTINCT dblink, c, split(dblink, ': ') as elements
                MERGE (e:Entity {uri: replace(elements[0], "'", "")})
                MERGE (c)-[r:appear]->(e)
                SET r.weight=elements[1]""")
        
        tx.run("""LOAD CSV WITH HEADERS FROM \"file:///enhanced_dataset.csv\" AS line FIELDTERMINATOR \",\" 
                MERGE(c:Document {id: line.id})
                WITH line, c
                UNWIND split(line.my_keyphrases, ',') AS keyphrase 
                WITH DISTINCT keyphrase, c, split(keyphrase, ': ') as elements
                MERGE (k:Keyphrase {name: trim(replace(elements[0], "'", ""))})
                MERGE (c)-[r:appear]->(k)
                SET r.weight=elements[1]""")
                
       
if __name__ == '__main__':
    db = NewDb("bolt://localhost:7687", "neo4j", "NEO4J_PASSWORD")
    db._prova_creazione()
    db.close()